In [ ]:
import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import dotenv

*** Acesso a PoloTrial (API)

In [ ]:
#ROTA GERAL

dotenv.load_dotenv(override=True)

session = requests.Session()

polotrial_url = os.getenv("POLOTRIAL_URL")
print(f" URL: {polotrial_url}")

payload = {
    "nome": os.getenv("POLOTRIAL_USER"),
    "password": os.getenv("POLOTRIAL_PASSWORD")
}

headers = {
    "Content-Type": "application/json"
}
session_url = f'{polotrial_url}/sessions'
print(f"session: {session_url}")
login_response = requests.request("POST", session_url, json=payload, headers=headers)

print(f"Status Code: {login_response.status_code}")

auth_cookie = login_response.cookies.get("userId")
endpoint_headers={
        "cookie": f"userId={auth_cookie}"
    }
print({endpoint_headers['cookie']})

# Mensagem de sucesso na optenção do cookie
if endpoint_headers['cookie']:
    print("Autenticação bem-sucedida. Cookie obtido.")
    print(endpoint_headers)
    print(polotrial_url)
else:
    print("Falha na autenticação. Verifique suas credenciais.")

In [ ]:
#ROTA PROTOCOLO

protocolo_url = f"{polotrial_url}/protocolo"
protocolos_ativos = [
    "161", #Aguardando iniciação
    "39", # Recrutamento aberto
    "9236", # Recrutamento aberto
    '9263', # Aguardando Ativação do Centro
    "393", # Aguardando Ativação do Centro
    '9294', # Fase Contratual
    '819', # Aguardando o Pacote Regulatório
    '739', # Aprovação Anvisa
    '6', # Aprovação Regulatória
    '912', # Em apreciação ética
    '9163'# Aprovado pelo CEP
]
 
df_protocolos = []
 
for co_status in protocolos_ativos:
    query_params ={
        "nested":"true",
        "status_protocolo": co_status
    }
   
    try:
        response = requests.request(
            "GET",
            protocolo_url,
            headers = endpoint_headers,
            params = query_params
        )
        response.raise_for_status()
        print(f"Status {co_status} - {response.status_code}")
        protocolo_json = response.json()
        if isinstance(protocolo_json, dict):
            protocolo_json = [protocolo_json]
        df_protocolos.append(pd.DataFrame(protocolo_json))
   
    except Exception as e:
        print(f"Erro ao buscar ID {co_status}: {e}")
 
if df_protocolos:
    protocolos_dataframe = pd.concat(df_protocolos, ignore_index=True)
    print(f"DataFrame de protocolos criado com {len(protocolos_dataframe)} registros.")
else:
    protocolos_dataframe = pd.DataFrame()
    print("Nenhum dado de protocolo foi recuperado.")

In [ ]:
#ROTA PESSOAS

pessoas_url = f"{polotrial_url}/pessoas"
query_params = {
    "nested":"true",
}
response = requests.request("GET", pessoas_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
pessoas = response.json()
pessoas_df = pd.DataFrame([pessoas])

In [ ]:
#ROTA AGENDA

agenda_url = f"{polotrial_url}/participante_visita"
query_params = {
    "nested":"true",
}
response = requests.request("GET", agenda_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
agenda = response.json()
agenda_df = pd.DataFrame([agenda])

In [ ]:
#ROTA PARTICIPANTES

particpantes_url = f"{polotrial_url}/participantes"
query_params = {
    "nested":"true",
}
response = requests.request("GET", particpantes_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
participantes = response.json()
participantes_df = pd.DataFrame([participantes])

In [ ]:
monitoria_url = f"{polotrial_url}/agenda"
query_params = {
    "nested":"true",
}
response = requests.request("GET", monitoria_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
monitoria = response.json()
monitoria_df = pd.DataFrame([monitoria])

*** Visualização e modelagem para extração de GCP (cadastros ativos)

In [ ]:
pessoas_df.head(5)

In [ ]:
gcp = pessoas_df[['dados_centro', 'dt_ultimo_certificado_gcp', 'ds_nome','dados_co_tipo_gn', 'dados_status']].copy()
gcp.loc[:, 'dados_centro_id'] = gcp['dados_centro'].apply(lambda x: x['id'] if x is not None else None)
gcp.loc[:, 'dados_centro_descricao'] = gcp['dados_centro'].apply(lambda x: x['descricao'] if x is not None else None)
gcp.loc[:, 'tipo_gn'] = gcp['dados_co_tipo_gn'].apply(lambda x: x['ds_descricao'] if x is not None else None)
gcp.loc[:, 'status'] = gcp['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

gcp_ativos = gcp.drop(['dados_centro', 'dados_centro_id', 'dados_co_tipo_gn', 'dados_status'], axis=1)
gcp_modelado = gcp_ativos[gcp_ativos['status'] == 'Ativo']

gcp_modelado.head(5)

In [ ]:
venc_gcp = gcp_modelado[gcp_modelado['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp["Cadastro de data"] = venc_gcp["dt_ultimo_certificado_gcp"].apply(
    lambda x: "Não há data cadastrada" if pd.isna(x) else "Data cadastrada"
)

venc_gcp.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função', 'status': 'Status do Cadastro'}, inplace=True)

nova_ordem = ['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]
venc_gcp = venc_gcp[nova_ordem]
venc_gcp = venc_gcp.sort_values(by="Centro")
venc_gcp

In [ ]:
hmcg = ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 'Leforte Liberdade', 'Clínica CardialMed']
filtro1 = venc_gcp['Centro'].isin(hmcg)
venc_gcp_hmcg = venc_gcp[filtro1]
venc_gcp_hmcg=venc_gcp_hmcg.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_hmcg

In [ ]:
rocio = ['Maternidade e Cirurgia Nossa Senhora do Rocio SA']
filtro2 = venc_gcp['Centro'].isin(rocio)
venc_gcp_rocio = venc_gcp[filtro2]
venc_gcp_rocio=venc_gcp_rocio.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_rocio

In [ ]:
iir_coord = ['Hospital das Clínicas de Itajubá', 'Saint-Beauté Clinique', 'Hospital Salvalus','Consultório Lopes e Sartorelli', 'Clínica Berger', 
            'QualiVida Higienópolis','Endolap Saúde' ]
filtro3 = venc_gcp['Centro'].isin(iir_coord)
venc_gcp_iir_coord = venc_gcp[filtro3] 
venc_gcp_iir_coord=venc_gcp_iir_coord.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_iir_coord

In [ ]:
envio_viviane = ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirão Preto',
                'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos RespiratÃ³rios',
                'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
                'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara',
                'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
                'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
                'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
                'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul ']
filtro4 = venc_gcp['Centro'].isin(envio_viviane)
venc_gcp_envio_viviane = venc_gcp[filtro4]
venc_gcp_envio_viviane=venc_gcp_envio_viviane.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_envio_viviane

In [ ]:
stacasa_santos = ['Santa Casa de Santos']
filtro5 = venc_gcp['Centro'].isin(stacasa_santos)
venc_gcp_stacasa_santos = venc_gcp[filtro5]
venc_gcp_stacasa_santos=venc_gcp_stacasa_santos.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_stacasa_santos

In [ ]:
capibaribe = ['Hospital do Capibaribe']
filtro6 = venc_gcp['Centro'].isin(capibaribe)
venc_gcp_capibaribe = venc_gcp[filtro6]
venc_gcp_capibaribe=venc_gcp_capibaribe.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_capibaribe

***Função para cálculo da data de vencimento de GCP.

In [ ]:
def verificar_vencimento_contratos(df_contratos, nome_centro, dias_para_vencimento=30):

    df_contratos['Assinatura do GCP'] = pd.to_datetime(df_contratos['Assinatura do GCP'], errors='coerce')

    hoje = datetime.today()
    limite_vencimento = hoje + timedelta(days=dias_para_vencimento)

    df_contratos['data_vencimento'] = df_contratos['Assinatura do GCP'] + timedelta(days=2 * 365)

    filtro_vencendo = (df_contratos['data_vencimento'] > hoje) & (df_contratos['data_vencimento'] <= limite_vencimento)

    filtro_sem_data = df_contratos['Assinatura do GCP'].isna()

    contratos_relevantes = df_contratos[filtro_vencendo | filtro_sem_data]

    if contratos_relevantes.empty:
        print(f"🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}.")
    else:
        print(f"⚠️{nome_centro} - Os profissionais abaixo listados não tem data de GCP na PoloTrial ou a data informada irá vencer nos próximos {dias_para_vencimento} dias:")
        print(contratos_relevantes[['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]])

    return contratos_relevantes

In [ ]:
contratos_hmcg = verificar_vencimento_contratos(venc_gcp_hmcg, "HMCG", dias_para_vencimento=30)
contratos_rocio = verificar_vencimento_contratos(venc_gcp_rocio, "Rocío", dias_para_vencimento=30)
contratos_iir_coord = verificar_vencimento_contratos(venc_gcp_iir_coord, "IRR COORD", dias_para_vencimento=30)
contratos_vivi = verificar_vencimento_contratos(venc_gcp_envio_viviane, "Envio Viviane", dias_para_vencimento=30)
contratos_stacsantos = verificar_vencimento_contratos(venc_gcp_stacasa_santos, "Santa Casa de Santos", dias_para_vencimento=30)
contratos_capiberibe = verificar_vencimento_contratos(venc_gcp_capibaribe, "Capiberibe", dias_para_vencimento=30)


***Função para envio dos emails de GCP

In [ ]:
css_hover = """
<style>
/* Reset básico */
table {
    border-collapse: collapse;
    width: 100%;
    font-family: Arial, Helvetica, sans-serif;
    font-size: 13px;
}

/* Cabeçalho */
th {
    background-color: #007bff;
    color: #ffffff;
    border: 1px solid #d0d0d0;
    padding: 8px;
    text-align: left;
    font-weight: bold;
}

/* Células */
td {
    border: 1px solid #d0d0d0;
    padding: 8px;
    text-align: left;
    vertical-align: middle;
    color: #000000;
}

/* Linhas alternadas (mais compatível que hover) */
tr:nth-child(even) {
    background-color: #f8f9fa;
}

/* Hover (funciona em Gmail/Web, ignorado no Outlook) */
tr:hover {
    background-color: #fff3cd;
}

/* Responsividade simples */
@media screen and (max-width: 600px) {
    table {
        font-size: 12px;
    }
}
</style>
"""

Variáveis de envio de email

In [ ]:
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME")
email_senha = os.getenv("EMAIL_PASSWORD")
enviar_para = os.getenv('ENVIAR_PARA')
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG')
destinatario_rocio =  os.getenv('DESTINATARIO_ROCIO')
destinatario_scs = os.getenv('DESTINATARIO_SCS')
destinatario_iir =  os.getenv('DESTINATARIO_IIR')
destinatario_vivi =  os.getenv('DESTINATARIO_VIVIANE')
destinatario_capiberibe =  os.getenv('DESTINATARIO_CAPIBERIBE')

In [ ]:
def enviar_email(destinatarios, df_tratado, nome_grupo):
    if df_tratado is None or df_tratado.empty:
        print("Não há contratos vencidos para enviar por e-mail.")
        return

    try:
        # Garante lista de destinatários
        if isinstance(destinatarios, str):
            destinatarios = [destinatarios]

        # Tabela HTML
        colunas = ['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]
        tabela_html = df_tratado[colunas].to_html(
            index=False,
            escape=False,
            border=0,
            justify="left",
            classes="table"
        )

        # Configuração do e-mail
        msg = MIMEMultipart("alternative")
        msg["From"] = email_usuario
        msg["To"] = email_usuario
        msg["Bcc"] = ", ".join(destinatarios)
        msg["Subject"] = f"[{nome_grupo}] GCP sem data registrada ou com vencimento próximo"

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>Vencimento de GCP - {nome_grupo}</h2>

                <p>Bom dia,</p>

                <p>
                    Segue abaixo relação de GCPs que não possuem data registradano sistema ou que apresentam vencimento próximo:
                </p>
                {tabela_html}
                <br>
                <p>
                    Este e-mail é gerado automaticamente a partir de informações inseridas na <strong>Polo Trial</strong>.
                </p>
                <p>
                    Qualquer dúvida, por favor, contate o <strong><u>time BI - SVRI</u></strong>.
                </p>
            </body>
        </html>
        """

        msg.attach(MIMEText(body, "html"))

        # Envio
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            server.send_message(msg)

        print(f"E-mail enviado com sucesso para o grupo: {nome_grupo}")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")


In [ ]:
enviar_email([destinatario_hmcg], contratos_hmcg, "HMCG")
enviar_email([destinatario_rocio], contratos_rocio, "Rocio")
enviar_email([destinatario_iir], contratos_iir_coord, "IIR")
enviar_email([destinatario_scs], contratos_stacsantos, "Sta. Santos")
enviar_email([destinatario_vivi], contratos_vivi, "Viviane")
enviar_email([destinatario_capiberibe], contratos_capiberibe, "Capiberibe")

***Modelagem para envio de Visitas de Seguimento

In [ ]:
protocolo = pd.DataFrame(protocolos_dataframe)
protocolo.head(5)

In [ ]:
centros = protocolo[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros.head(5)

In [ ]:
seguimento = agenda_df[['dados_participante', 'data_estimada', 'dados_status', 'dados_visita']].copy()

seguimento.loc[:, 'dados_participante_id'] = seguimento['dados_participante'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'id_participante'] = seguimento['dados_participante'].apply(lambda x: x['id_participante'] if x is not None else None)
seguimento.loc[:, 'dados_protocolo'] = seguimento['dados_participante'].apply(lambda x: x['dados_protocolo'] if x is not None else None)

seguimento.loc[:, 'dados_status_id'] = seguimento['dados_status'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'ds_descricao'] = seguimento['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
seguimento.loc[:, 'ds_nome_visita'] = seguimento['dados_visita'].apply(lambda x: x['ds_nome_visita'] if x is not None else None)

seguimento.loc[:, 'apelido_protocolo'] = seguimento['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)
seguimento['data_estimada'] = pd.to_datetime(seguimento['data_estimada'], errors='coerce')
seguimento.head(5)

In [ ]:
seguimento_tratado = seguimento.drop(['dados_participante','dados_status','id_participante','dados_status_id'], axis=1)
seguimento_tratado.loc[:, 'id'] = seguimento['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
seguimento_tratado.head(5)

In [ ]:
seguimentos = pd.merge(centros, seguimento_tratado, on='id', how='inner')
seguimentos

In [ ]:
seguimentos['apelido_protocolo_x'] = seguimentos['apelido_protocolo_x'].str.strip()
visitas_filtrado = seguimentos[seguimentos['ds_descricao'].str.contains('Pendente')]
nova_ordem = ["apelido_centro", "dados_participante_id", "ds_nome_visita","data_estimada","ds_descricao"]
visitas_reordenado = visitas_filtrado[nova_ordem].rename(
    columns={
        'apelido_centro':'Estudo/Centro',
        'dados_participante_id': 'ID Participante',
        'ds_nome_visita': 'Tipo Visita',
        'data_estimada':'Data Estimada',
        'ds_descricao':'Status'
    }
)
visitas_reordenado['Data Estimada'] = pd.to_datetime(visitas_reordenado['Data Estimada'], errors='coerce')
visitas_reordenado

In [ ]:
datas_visitas = visitas_reordenado.copy()
datas_visitas['Data Estimada'] = pd.to_datetime(datas_visitas['Data Estimada'], errors='coerce')
datas_visitas = visitas_reordenado.dropna(subset=["Data Estimada"])
datas_visitas = datas_visitas.sort_values(by= "Data Estimada", ascending=True)
datas_visitas = datas_visitas[~datas_visitas["Tipo Visita"].isin(["Unscheduled", "Triagem", "End of Study"])]
valores_contagem = datas_visitas['Tipo Visita'].value_counts()
datas_visitas

In [ ]:
def verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20):
    df = datas_visitas.copy()

    df['Data Estimada'] = (
        pd.to_datetime(df['Data Estimada'], errors='coerce')
        .dt.tz_localize(None)   # 👈 REMOVE UTC
    )

    hoje = pd.Timestamp.today().normalize()
    limite_visita = hoje + pd.Timedelta(days=dias_para_visita)

    visitas_futuras = df[
        (df['Data Estimada'] >= hoje) &
        (df['Data Estimada'] <= limite_visita)
    ]

    return visitas_futuras

visitas_20_dias = verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20)

In [ ]:
def enviar_emails(próximas_visitas):
    global enviar_para  # Acessa a variável global 'enviar_para'
    
    try:
        if próximas_visitas is None or próximas_visitas.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        colunas_esperadas = {'Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada', 'Status'}
        if not colunas_esperadas.issubset(próximas_visitas.columns):
            print("Erro: DataFrame não contém todas as colunas esperadas.")
            return

        if not all([smtp_server, email_usuario, email_senha, email_port]):
            print("Erro: Configurações de e-mail estão incompletas.")
            return

        # Validação de e-mails com regex
        email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        if isinstance(enviar_para, list):
            enviar_para = [email.strip() for email in enviar_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            enviar_para = []

        # Criação da tabela HTML
        tabela_html = próximas_visitas[list(colunas_esperadas)].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        # Montagem da mensagem
        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Seguimento - próximos 20 dias"
        
        # Define o campo BCC apenas se houver destinatários válidos
        if enviar_para:
            msg['Bcc'] = ', '.join(enviar_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de seguimento programadas para os próximos 20 dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        # Envio de e-mail
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            
            # Lista final de destinatários
            destinatarios = [email_usuario] + enviar_para
            if destinatarios:
                server.sendmail(email_usuario, destinatarios, msg.as_string())
                print("E-mail de visitas pendentes enviado com sucesso!")
            else:
                print("Nenhum destinatário válido. E-mail não enviado.")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(visitas_20_dias)

**** Visitas de Monitoria

In [ ]:
monitoria_df

In [ ]:
monitoria = monitoria_df[['dados_protocolo', 'dados_tipo', 'dados_status', 'data_estimada_filter']].copy()

monitoria.loc[:, 'dados_protocolo_id'] = monitoria['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'apelido_protocolo'] = monitoria['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)

monitoria.loc[:, 'dados_tipo_id'] = monitoria['dados_tipo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'dados_tipo_tipo'] = monitoria['dados_tipo'].apply(lambda x: x['tipo'] if x is not None else None)

monitoria.loc[:, 'dados_status_id'] = monitoria['dados_status'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'ds_descricao'] = monitoria['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

monitoria = monitoria.drop(['dados_protocolo','dados_status', 'dados_tipo_id','dados_status_id'], axis=1)
monitoria.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)

monitoria.head(2)

In [ ]:
monitorias = pd.merge(centros, monitoria, on='id', how='inner')
monitorias

In [ ]:
monitorias_filtrado = monitorias[monitorias['dados_tipo_tipo'].str.contains('Monitoria')]
monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()
monitorias_filtrado

In [ ]:
def verificar_monitorias_pendentes():
    df = monitorias_filtrado.copy()

    df['data_estimada_filter'] = (
        pd.to_datetime(df['data_estimada_filter'], errors='coerce')
        .dt.tz_localize(None)
    )

    hoje = pd.Timestamp.today().normalize()

    monitorias_pendentes = df[
        (df['ds_descricao'] == 'Pendente') &
        (df['data_estimada_filter'] >= hoje)
    ].sort_values('data_estimada_filter')

    print(
        monitorias_pendentes[
            ['dados_tipo_tipo', 'data_estimada_filter',
             'apelido_protocolo_x', 'ds_descricao', 'apelido_centro']
        ]
    )

    return monitorias_pendentes


    schedule.every().monday.at("09:00").do(verificar_monitorias_pendentes)
monitorias_pendentes = verificar_monitorias_pendentes()


In [ ]:
# Variavel de envio do email 

envio_para = os.getenv('ENVIO_PARA')

In [ ]:
monitorias_pendentes.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_tipo_tipo': 'Monitoria', 'data_estimada_filter': 'Data Estimada','ds_descricao': 'Status'}, inplace=True)

monitorias_pendentes

In [ ]:
monitoria_envio = monitorias_pendentes.drop(columns=['apelido_protocolo_x', 'co_externo', 'dados_tipo', 'apelido_protocolo_y'],errors='ignore')

monitoria_envio = monitoria_envio[
    [
        'Monitoria',
        'id',
        'numero_protocolo',
        'Estudo/Centro',
        'Data Estimada',
        'Status'
    ]
]
monitoria_envio

In [ ]:
def enviar_emails(monitoria_envio):
    global envio_para
    try:
        if monitoria_envio is None or monitoria_envio.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        tabela_html = monitoria_envio[['Estudo/Centro', 'Monitoria', 'Data Estimada', 'Status']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Monitoria Pendentes"

        # Validação da lista de destinatários
        if isinstance(envio_para, list):
            # Expressão regex para validar e-mails
            email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
            envio_para = [email.strip() for email in envio_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            envio_para = []

        # Verifica se há destinatários antes de enviar
        destinatarios = [email_usuario] + envio_para
        if not destinatarios:
            print("Nenhum destinatário válido encontrado. O e-mail não foi enviado.")
            return

        # Somente adiciona Bcc se houver destinatários válidos
        if envio_para:
            msg['Bcc'] = ', '.join(envio_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de monitoria programadas para os pr&oacute;ximos dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            # Envio corrigido para passar apenas destinatários válidos
            server.sendmail(email_usuario, destinatarios, msg.as_string())

        print("E-mail de visitas pendentes enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(monitoria_envio)